In [ ]:
// Import the required libraries
#r "nuget: Plotly.NET.Interactive, 3.0.2"
#r "nuget: FSharp.Stats, 0.4.7"
#r "nuget: FsODE, 0.0.1"

// ... and open the required modules
open FsODE
open Plotly.NET
open FSharp.Stats

# Tag 2 - Modellierung von Wachstum

### Inhalt

- Theorie
  - Warum ist es wichtig Zellwachstum zu quantifizieren?
- Einfaches Wachstumsmodell als Differenzialgleichung
  - Simulation des Modells und Visualisierung der geschlossener Form
- Exkurs Logarithmus
  - Warum ist der Logarithmus bei exponentiellem Wachstum sinnvoll? 
- Erweiterung des einfachen Wachstumsmodells zu logistischem Wachstum
  - Simulation des Modells und Visualisierung der geschlossener Form
- Biologisches Beispiel
  - Modelfitting logistisches Wachstum
- Anleitung Generationszeitbestimmung



## Theorie

Zu den ältesten und 'vermeintlich' einfachsten Experimenten gehört die Analyse von Wachstum, sei es von Bakterien, Hefen, Zellen in Zellkultur, ganzen Organismen oder Populationen. Wachstum kann je nach untersuchtem System durch verschiedene Parameter beschrieben werden. Steht der Organismus im Fokus, so sind Gewicht, Länge oder Größe von Interesse. Bei Populationen geht es meist um eine Zellzahl pro definiertem Volumen oder Fläche (Populationsdichte).
Im Labor können verschiedene Wachstumsbedingungen künstlich erzeugt werden und damit der Einfluss jedes einzelnen Faktors - getrennt oder kombiniert - untersucht werden. Zum Beispiel können Bakterien statt auf Standardmedium mit Glucose auf Medium mit Galactose angezogen werden. Eine eventuell nötige Umwandlung von Galactose in Glucose, oder aufwendigere metabolische Reaktionen führen hier zu langsamerem Wachstum. Dies lässt sich in der Regel durch einfache Absorptionsmessungen im Photometer beweisen. Ein vermindertes Wachstum geht hier mit einer langsameren Trübung (niedrigere optische Dichte) des Mediums einher was auf einem Graphen leicht visualisiert werden kann.

<center>
<img src="../../images/day2/02_Growth_01.png" width=40% />
</center>

Leider kann der Wachstumsunterschied mit dem oben gezeigten Datensatz nicht quantifiziert werden. Zwar kann die zeitliche Differenz zwischen zwei Punkten gleicher optischer Dichte bestimmt werden (Kreuze in Abbildung 1), jedoch ist diese Differenz sowohl stark von den gewählten Messzeitpunkten, als auch von den vorliegenden (unbekannten) Wachstumsphasen abhängig. Außerdem verläuft die Trübung nicht linear. Das bedeutet, dass ich bei einer Zellverdopplung nicht automatisch auch die Absorption verdoppelt.

Wesentlich aussagekräftiger ist eine Bestimmung der absoluten Zellkonzentration mit anschließendem Fitting eines bekannten Wachstumsmodells. Die so ermittelten Parameter können für quantitative und vergleichende Analysen von z.B. Glucose- und Galactosemedium verwendet werden. Das einfachste Wachstumsmodell wollen wir uns im Folgenden anschauen.  

## Einfaches Wachstumsmodell

Wachstum, also die Änderung der Zellzahl über die Zeit, ist proportional zur Zellzahl _N_. Der Proportionalitätsfaktor ist _r_. Je höher die Zellzahl, desto höher ist die Rate mit der die Zellzahl wächst (Diese Begründung bitte kurz sacken lassen).

_Aufgabe 2.1: Wie sieht die beschriebene Differentialgleichung aus?_

a) $\frac{dN}{dt} = \frac{r}{N}$

b) $N(t) = rtN$

c) $N(t) = \frac{rN}{t}$

d) $\frac{dN}{dt} = rN$



Antwort (Editor mit Doppelklick auf diese Zelle öffnen):

_Aufgabe 2.2: Beweisen Sie, dass $N(t) = N_0 *e^{rt}$ eine Lösung für die oben definierte Differentialgleichung ist._

Hinweis: Leiten sie die obenstehende geschlossene Form der Wachstumsgleichung nach _t_ ab und setzen Sie _N_ und die Ableitung in die Differenzialgleichung ein. Versuchen Sie diese Aufgabe innerhalb von 10 Minuten zu lösen. Fahren Sie ansonsten mit dem nächsten Abschnitt fort.



Nachdem das Wissen über Ableitungen aufgefrischt wurden, soll die Wachstumsfunktion nun simuliert werden. Dafür wird die Differenzialgleichung als Modell formalisiert und mit folgenden Werten parametrisiert:

$$r = 0.3$$

$$N(t0) = 100$$

$$t0 = 0.0$$

_Aufgabe 2.3: Setzen Sie die gegebene Differenzialgleichung und Parametern in das Modell ein._

In [ ]:
// First set the model context that remembers the solver method and it´s options
let modelContext = 
    OdeContext(
        OdeSolverMethod.RK546M,
        OdeSolverOptions(
            StepSize=0.42
        )
    )

// growth rate
let r = 0.3         //aufgabe

// growth model
let dN_dt : SimpleModel =     
    fun N t ->
        r * N       //aufgabe

// Initial condition
let t0 = 0         //aufgabe 
let N0 = 100.0     //aufgabe

_Aufgabe 2.4: Visualisieren Sie das Modell anhand der ersten 10 Datenpunkte und beschriften Sie die x- und y-Achse._

In [ ]:
// Simulate the model function
let sim_dy_dx = 
    modelContext.OdeInt(t0,N0,dN_dt) //aufgabe
    |> SolPoints.take 10 
    |> SolPoints.memorize

// Plot the numerical solution
sim_dy_dx
|> SolPoints.toPoints 1
|> Chart.Point
|> Chart.withTitle("Abbildung 2 - Simulation simple growth model")
|> Chart.withXAxisStyle("Time") //aufgabe
|> Chart.withYAxisStyle("Population size") //aufgabe

_Aufgabe 2.5: Können Sie anhand des Graphen abschätzen wo lag und log-Phase liegen?_

Antwort:

_Aufgabe 2.6: Schätzen Sie grob die Steigung zum Zeitpunkt t=1 ab. Entpricht Sie der theoretischen Steigung aus der Differentialgleichung (m=rN)?_

Antwort:

Die Lösung des Modells hat einen möglichen Funktionsverlauf simuliert der den Bedingungen der Differenzialgleichung gerecht wird. Im Folgenden wollen wir die Simulation der Differenzialgleichung mit den tatsächlichen Funktionswerten vergleichen. Dafür muss die Funktion aus Aufgabe 2.2 erstellt werden. Sie hängt ausschließlich von Parameter t ab. Exponenten zur Basis _e_ können durch exp(x) erstellt werden.

_Aufgabe 2.7: Formulieren Sie die Wachstumsfunktion aus Aufgabe 2.2._

In [ ]:
let y_exact =
    fun t -> 
        N0*exp(r*t) //aufgabe


[
    sim_dy_dx
    |> SolPoints.toPoints 1
    |> Chart.Spline 
    |> Chart.withTraceInfo("simulation");
    
    sim_dy_dx
    |> SolPoints.map (fun p -> (p.x, y_exact p.x))
    |> Chart.Point
    |> Chart.withTraceInfo("exact");

]
|> Chart.combine
|> Chart.withTitle("Abbildung 3 - Simple growth model comparison")
|> Chart.withXAxisStyle("Time")
|> Chart.withYAxisStyle("Population size")

Die Simulation der Differenzialgleichung ist gleich zur exakten Lösung der geschlossenen Form.



Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen:


### Expertenaufgaben
- Integrieren Sie $\frac{dN}{dt}=rN$ zu $N(t) = N_0 *e^{rt}$

___



## Exkurs Logarithmus


Wie Sie vielleicht gemerkt haben, wächst die Population in obigem Beispiel zu jedem Zeitpunkt mit der gleichen Rate exponentiell. Die Steigungsänderung hängt nie von t, sondern nur von der gerade vorhandenen Zellzahl "N" und der Konstante "r" ab (vgl. 2.2). In den Abbildung wirkt es jedoch, als würden sich die Zellen mit fortschreitender Zeit immer schneller teilen. Um diesen Effekt zu vermeiden **müssen** Zellzahlen grundsätzlich vor einer Wachstumsanalyse logarithmiert werden. Diese Transformation führt dazu, dass exponentielle Verhaltensweisen visuell korrigiert werden.

Ein Beispiel:

Ein Bakterium teilt sich jede Stunde genau ein mal. Nach der ersten Stunde sind es 2, nach der zweiten 4, nach der dritten 8 usw. Dieses Verhalten lässt sich durch $f(t)=2^t$ wie im Folgenden gezeigt darstellen.

_Aufgabe 2.8: Formulieren Sie die Funktion f(t), die dieses Verhalten widerspiegelt._


In [ ]:
let f t = 
    2.0**t //ersetzen

let xyWerte = 
    [0..24] 
    |> Seq.map (fun t -> 
        let xWert = t
        let yWert = f t
        xWert,yWert
        ) 

Chart.Line xyWerte
|> Chart.withXAxisStyle("Time [h]")
|> Chart.withYAxisStyle("Bakterienzahl")
|> Chart.withTitle("Abbildung 4 - Logarithmus I")

Allein durch das Betrachten einer exponentielle Funktion wird nicht klar, was zwischen Stunde 0 und Stunde 15 passiert. Um dafür einen Eindruck zu erhalten werden die y-Werte logarithmiert und erneut visualisiert.

In [ ]:
let werteTransformation (transformation: float -> float) = 
    xyWerte
    |> Seq.map (fun (x,y) -> 
        x,transformation y
        ) 

let log2Transform = werteTransformation log2
let log10Transform = werteTransformation log10
let lnTransform = werteTransformation log


[
Chart.Line (log2Transform,Name="log2")
Chart.Line (log10Transform,Name="log10")
Chart.Line (lnTransform,Name="ln")
]
|> Chart.combine
|> Chart.withXAxisStyle("Time [h]")
|> Chart.withYAxisStyle("logX(Bakterienzahl)")
|> Chart.withTitle("Abbildung 4 - Logarithmus II")

Eine log-transformation, hat auf hohe Werte einen deutlich größeren Einfluss als auf kleine Werte. Die Interpretation des Verlaufs ist nun an jedem beliebigen x-Wert möglich, ohne dass man in den Plot "reinzoomen" muss. Aus exponentiellem Verhalten wird durch log-Transformation eine Gerade. Es wird deutlich, dass egal welche log Transformation angewandt wurde (log zur Basis 2, 10 oder der eulerischen Zahl (natürlicher Logarithmus)) Funktionen zu Linien mit unterschiedlichen Steigung werden.

_Aufgabe 2.9:_

  _a) Wie groß muss der y-Achsenabschnitt sein, um eine Verdopplung der Bakterien zu erhalten?_ 

  _b) Um Zellverdopplungen zu analysieren, welchen Logarithmus würden Sie nutzen um die visuelle Analyse so einfach wie möglich zu machen und warum?_ 

  _c) Welcher Logarithmus wird bei pH-Werten genutzt und warum könnte dies als sinnvoll erachtet werden um zum Beispiel pH 2 direkt mit pH 5 vergleichen zu können?_

a)

  - Antwort log2:   

  - Antwort log10:  

  - Antwort ln:     

b)

  - Antwort:

c)

  - Antwort:

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen:


### Expertenaufgaben
- Wie groß ist der y-Achsenabschnitt im log space (log2, ln und log10) um von y=100. zu y=10000. zu kommen? Lösen Sie die Aufgabe rechnerisch.


___


## Biologisches Beispiel

Von einer Bakterienkultur wurde in regelmäßigen Abständen die Zellzahl bestimmt. Die gemessenen Werte können Sie aus untenstehender Tabelle entnehmen


|Zeitpunkt $(h)$|Zellzahl $(\frac{10^6}{ml})$|
|--|--|
|0.0|17|
|0.5|16.5|
|1.0|11|
|1.5|14|
|2.0|27|
|2.5|40|
|3.0|120|
|3.5|300|
|4.0|450|
|4.5|1,200|
|5.0|2,700|
|5.5|5,000|
|6.0|8,000|
|6.5|11,700|
|7.0|12,000|
|7.5|13,000|
|8.0|12,800|

In der folgenden Zelle wird die Zellzahl als Funktion der Zeit visualisiert.

In [ ]:
let time = [|0. .. 0.5 .. 8.|]

let cellCount =
    [|
    17000000.;16500000.;11000000.;14000000.;27000000.;40000000.;120000000.;
    300000000.;450000000.;1200000000.;2700000000.;5000000000.;
    8000000000.;11700000000.;12000000000.;13000000000.;12800000000.
    |]

let cellCountLog2 = 
    Array.map log2 cellCount

let chartOrig = 
    Chart.Point(time,cellCount)
    |> Chart.withTraceInfo "original data"
    |> Chart.withYAxisStyle "cells*10^6/ml"

let chartLog =
    Chart.Point(time,cellCountLog2)
    |> Chart.withTraceInfo "log count"
    |> Chart.withXAxisStyle "time (h)" 
    |> Chart.withYAxisStyle "log2(cells*10^6/ml)"
    
let growthChart = 
    [chartOrig;chartLog] 
    |> Chart.Grid(2,1)
    |> Chart.withTitle("Abbildung 5 - Beispiel I")

growthChart

Es ist ein deutlicher visueller Unterschied im Funktionsverlauf zu erkennen. Während im nicht-transformierten Diagramm der Zeitpunkt um 6 Stunden noch stark ansteigend (exponentielles Wachstum) erscheint, wird erst im transformierten Diagramm klar, dass die exponentielle Phase von Zeitpunkt 2 - 5 andauert und sich bei Stunde 6 schon am Übergang zum stationären Wachstum befindet. Durch die Abhängigkeit von Steigung und Zellzahl werden die echten Wachstumsphasen in untransformierten Daten kaschiert. Daher sollten Wachstumsdaten in der Regel im log-space angezeigt und analysiert werden. 

Wichtig: Im Folgenden wird immer davon ausgegangen, dass die Daten log transformiert wurden bevor die Kurven modelliert werden!

<center>
<img src="../../images/day2/02_Growth_02.png" width=50% />
</center>

**Abbildung 6: Schema einer klassischen Wachstumskurve**

Wie im Schema gezeigt ist es sinnvoll die Daten zu fitten um bestimmte Parameter wie y0, ymax,  $\mu max$ und $\lambda$ ermitteln zu können. Ein einfaches Modell für exponentielles Wachstum haben wir bereits im ersten Abschnitt kennen gelernt. Würden wir auf die transformierten Daten (Abbildung 5 unten) nun jedoch das einfache Wachstumsmodell anwenden, dann würde es nicht zu einem sigmoidalen Verlauf kommen, sondern die Kurve würde immer weiter exponentiell ansteigen. Es gibt einige Wachstumsmodelle, die diese Limitationen nicht aufweisen (http://www.pisces-conservation.com/growthhelp/index.html). Gesucht wird eine Erweiterung des einfachen Modells $\frac{dN}{dt} = rN$, die das Wachstum sowohl in der lag-, als auch in der stationären Phase begrenzen kann, also ein Faktor, der abhängig von der maximalen Zellzahl die Wachstumsgeschwindigkeit (growth rate) beschränkt:

$\frac{dN}{dt}=rN(1-\frac{N}{k})$

wobei $K$ die so genannte carrying capacity, also die Maximalanzahl von Zellen darstellt. Dieses Modell ist das Verhulst-Wachstumsmodell und beschreibt eine logistische Funktion mit unterer und oberer Asymptote.

_Aufgabe 2.10: Erstellen Sie ein Modell der angepassten Differenzialgleichung, einem Wachstumsparameter "r" von 0.3, einer Maximalanzahl "k" von 20 und einer Startzellzahl von 1._





In [ ]:
// parameter
let r = 0.3
let k = 20.

// growth model
let verhulst : SimpleModel =     
    fun N t ->
        N * r * (1. - N / k)

// Initial condition
let t0 = 0
let N0 = 1.

In [ ]:
// Simulate the model function
let sim_dy_dx = 
    modelContext.OdeInt(t0,N0,verhulst)
    |> SolPoints.take 15 
    |> SolPoints.memorize

// Plot the numerical solution
sim_dy_dx
|> SolPoints.toPoints 1
//|> Seq.map (fun (x,y) -> x, log2 y)
|> Chart.Point
|> Chart.withTitle("Abbildung 7 - Simulation verhulst growth model")
|> Chart.withXAxisStyle("Time [h]")
|> Chart.withYAxisStyle("log2(Zellzahl)")

Die lag- und stationäre Phase sind hier deutlich erkennbar. Dieses Modell soll nun an die gegebenen Daten aus dem biologischen Beispiel gefittet werden. Dazu suchen wir die Integration der Differenzialgleichung und versuchen diese Funktion durch Änderung der Parameter nah an die gegebenen logarithmierten Daten zu fitten. Ein Integral lautet:

$$N(t)=lmin + \frac{lmax - lmin}{1+e^{(k-t)/d}}$$

mit

$lmin$ = untere asymptote

$lmax$ = obere asymptote/Zellmaximum

$k$ = x-Wert des Wendepunktes

$d$ = Steilheit

Zur Veranschaulichung wird die Verhulst-Funktion mit 4 verschiedene Parametersets visualisiert.

In [ ]:
let verhulstFunktion lmin lmax (k:float) d t=
    lmin + (lmax - lmin)/(1.+ exp((k-t)/d))

let plotVerhulst lmin lmax  k d =
    let data=
        [0..40]
        |> Seq.map (fun x -> x,verhulstFunktion lmin lmax k d x)
    let chart = 
        Chart.Line(data,Name=sprintf "min:%.0f max:%.0f k:%.0f d:%.0f" lmin lmax k d)
    chart

[
plotVerhulst 0. 10. 4. 1.
plotVerhulst 0. 10. 20. 1.
plotVerhulst 0. 10. 20. 3.
plotVerhulst 5. 20. 10. 3.
]
|> Chart.combine
|> Chart.withLegendStyle(X=0.6,Y=0.01)
|> Chart.withTitle("Abbildung 8 - Verschiedene logistische Funktionen")
|> Chart.withXAxisStyle("Time [h]")
|> Chart.withYAxisStyle("log2(Zellzahl)")

Wie genau das Fitting der Funktion an die echten Daten geschieht, ist nicht Lernziel diese Praktikums, daher nur eine kurze Erläuterung. Mehr Details finden Sie in *Vorlesung 13 - Principles of model fitting*:

Ein sogenannter Solver benötigt die Daten und das Modell, welches auf diese Daten appliziert werden soll. Der Solver versucht die Parameter des Modells so lange zu optimieren, bis der Abstand zwischen dem Modell und den echten Daten minimal geworden ist. Anschließend kann man die optimierten Parameter abfragen und mit diesen weiter arbeiten. 

In [ ]:
open System
open FSharp.Stats
open FSharp.Stats.Fitting.NonLinearRegression
open FSharp.Stats.Fitting.LinearRegression

let getVerhulstParams timePoints logdata min max k d = 
    LevenbergMarquardt.estimatedParams
        Table.GrowthModels.verhulst4Param
        (createSolverOption 0.001 0.001 10000 [|max;k;d;min|])
        0.1
        10.
        timePoints
        logdata

let verhulstParams =
    //getVerhulstParams time cellCountLog2 23. 35. 3.5 1.
    getVerhulstParams time cellCountLog2 23.391 33.598 4. 1.

printfn "Für die gegebenden Daten sind die optimalen Parameter des 4-Parameter Verhulstmodells:
lmin = %.3f 
lmax = %.3f
k = %.3f
d = %.3f" verhulstParams.[3] verhulstParams.[0] verhulstParams.[1] verhulstParams.[2]



Der folgende Codeblock dient nur der Visualiserung der Daten mit gefittetem Modell.

In [ ]:
let fittedValuesVerhulst =
    /// The parameter were determined locally for saving time during build processes
    let f =  Table.GrowthModels.verhulst4Param.GetFunctionValue verhulstParams
    [time.[0] .. 0.1 .. Seq.last time]
    |> Seq.map (fun x -> 
        x,f x
        ) 
    |> Chart.Line

let fittedChartVerhulst = 
    fittedValuesVerhulst 
    |> Chart.withXAxisStyle "time (h)" 
    |> Chart.withYAxisStyle "log2(cells/ml)"
    |> Chart.withTraceInfo "verhulst"

let fittedChartVerhulstS = 
    [
        Chart.Point(time,cellCountLog2)
        |> Chart.withTraceInfo "log2 count"
        fittedChartVerhulst
    ]
    |> Chart.combine
    |> Chart.withXAxisStyle "time (h)" 
    |> Chart.withYAxisStyle "log2(cells/ml)"

fittedChartVerhulstS

Es folgt eine kurze Besprechung. Falls Sie schnell vorangekommen sind, können sie bis zur Besprechung folgende Aufgaben lösen:


### Expertenaufgaben
- Teilen Sie die Zeitpunkte in lag, log und stationäre Phase ein.
- Wie sind im gezeigten Fall die Werte der unteren und oberen Asymptote? Geben Sie die Werte als untransformierte Zellzahl an.

___

## Verdopplungszeit-Bestimmung

Wie bereits herausgefunden, ist eine Zellverdopplung im log2 space gleichzusetzen mit einem y-Achsenabschnitt von 1. Um nun die Generations/Verdopplungs-Zeit zu bestimmen muss die Zeit bestimmt werden in dem sich die log(Zellzahl) um 1 erhöht. Dies kann durch die maximale Steigung ($\mu max$) des gefitteten Modells erreicht werden. Diese entspricht dem schnellstmöglichen Wachstum und ist naturgemäß am Wendepunkt (inflection point) der sigmoidalen Kurve. Diese Steigung wird auch als "maximal growth rate" oder "specific growth rate" bezeichnet und wird gerne genutzt um Unterschiede zwischen Experimenten zu quantifizieren. 

Aber Achtung:

> Two definitions for this parameter [$\mu max$] are in current use. One is based on the inflection of the slope of the growth curve in the exponential phase (Zwietering et al., 1990) whilst the other is taken to be the growth rate at infinite dilution (Baranyi et al., 1993). - _Perni et al., 2004_


_Aufgabe 2.11: Schätzen Sie ab, wo sich der Wendepunkt - also der Punkt maximaler Steigung - befindet._

Antwort:

Da im Verhulstmodell ein Parameter schon optimiert wurde, der genau den x-Wert des Wendepunktes definiert, muss man nur noch diesen Wert (k = 3.525) in die Ableitung einsetzen um die maximale Steigung zu erhalten

$$N(t)=lmin + \frac{lmax - lmin}{1+e^{(k-t)/d}}$$


$$N'(t)=\frac{(lmax-lmin)e^{(k-t)/d}}{d(e^{(k-t)/d}+1.)^2}$$

optimierte Parameter aus Model-Fitting:

$lmin = 23.578$

$lmax = 33.987$

$k = 3.997$

$d = 0.956$

$$N(3.997) = 28.782$$

$$N'(3.997) = \frac{lmax - lmin}{4d} = 2.72$$


The maximal slope was determined to be 2.72. 

In [ ]:
let generationtimeVerhulst (verhulstParameters:vector) =
    let lmax = verhulstParameters.[0] //lmax = 33.696
    let k    = verhulstParameters.[1] //k = 3.525
    let d    = verhulstParameters.[2] //d = 1.073
    let lmin = verhulstParameters.[3] //lmin = 22.706
    let gradientFunctionVerhulst t =
        let numerator = (lmax-lmin)*Math.Exp((k-t)/d)
        let denominator = d*(Math.Exp((k-t)/d)+1.)**2.
        numerator / denominator
    let maximalSlope =
        gradientFunctionVerhulst k
    log2(2.) / maximalSlope

let generationVerhulst = 
    sprintf "The generation time (Verhulst) is: %.1f min" (generationtimeVerhulst verhulstParams * 60.)

printfn "%s" generationVerhulst

Die bestimmte Verdopplungszeit von 22 Minuten kann nun genutzt werden um sie gegen Verdopplungszeiten anderer Experimente zu vergleichen.

### Anleitung Generationszeitbestimmung
1. Plotten Sie die log2 transformierten Daten als xy Diagramm.
2. Wenn ein Wachstumsmodell und Solver vorhanden ist, fitten Sie das Modell an die Daten und isolieren sie $\mu max$.
3. Wenn kein Wachstumsmodell vorhanden ist, schätzen Sie den linearen Bereich der Wachstumskurve.
    - Erstellen Sie einen Ausgleichsgerade von diesem Bereich und isolieren Sie die Steigung $\mu max$.
4. Um die Generationszeit zu bestimmen rechnen Sie $\frac{log_2 2.}{\mu max} = \frac{1}{\mu max}$.


### Take home messages
- Die logistische Wachstumsfunktion $N(t)=lmin + \frac{lmax - lmin}{1+e^{(k-t)/d}}$ hat niemand einfach entwickelt, sondern es ist ein Integral eines leicht nachzuvollziehenden Wachstumsmodells $\frac{dN}{dt}=rN(1-\frac{N}{k})$
- Wenn Wachstum untersucht werden soll, sollte man sich den zeitlichen Verlauf (fast) immer mit log-transformierten Zellkonzentrationen anschauen.
- Bei log2 transformierten Daten ist ein y-Achsenabschnitt von 1 gleichzusetzen mit einer Verdopplung der Zellzahl, bei einer log10 Transformation ist es eine Verzehnfachung.
- Die Verdopplungszeit hängt ausschließlich von der Steigung der log-phase ($\mu max$ oder _maximal growth rate_) ab. Sie wird berechnet mit $\frac{log_x 2}{\mu max}$ wobei x die gewählte Basis ist, mit der die Zellzahlen transformiert wurden.
- Die Steigung kann entweder durch manuelle Selektion der log phase und anschließendes Fitten einer Gerade geschehen, oder durch fitten eines Wachstumsmodells ([Gompertz/Richards/Verhulst](https://fslab.org/FSharp.Stats/GrowthCurve.html)).



### References
- Maier R., Pepper I., Chapter 3 - Bacterial Growth, Environmental Microbiology (Third Edition), 2015, https://doi.org/10.1016/B978-0-12-394626-3.00003-X
- Perni S., Andrew P., Shama G., Estimating the maximum growth rate from microbial growth curves: definition is everything, Food Microbiology, 2005, https://doi.org/10.1016/j.fm.2004.11.014
- Venn B., Schneider K., Weil L., Zimmer D. and Mühlhaus T., fslaborg/FSharp.Stats: Release 0.4.7, 2022, Growth curve documentation: https://fslab.org/FSharp.Stats/GrowthCurve.html
- Growth II, version 2.3.6.71, PISCES Conservation Ltd United Kingdom



### Expertenaufgaben
- Wie ist die Verdopplungszeit? Die Zeipunkte sind als Tage angeben.
  - a) $\mu max = 2.5$; Basis der log-Transformation: 2
  - b) $\mu max = 1.0$; Basis der log-Transformation: e
  - c) $\mu max = 0.7$; Basis der log-Transformation: 10
  - d) $\mu max = 10$; Basis der log-Transformation: e
  - e) $\mu max = 10$; Basis der log-Transformation: 2
  - f) $\mu max = 10$; Basis der log-Transformation: 10


